# Bayesian Inference in Python

**Author: Jessica Cervi**


## Activity Overview

This activity is designed to consolidate your knowledge about Bayesian methods and to teach you how to use them in `Python` using the `sklearn` library.

We'll begin by seeing a simple, from scratch, implementation of the algorithm, and we will conclude by looking at how to run the same model in `sklearn`.


This activity is designed to help you apply the machine learning algorithms you have learned using packages in `Python`. `Python` concepts, instructions, and starter code are embedded within this Jupyter Notebook to help guide you as you progress through the activity. Remember to run the code of each code cell prior to submitting the assignment. Upon completing the activity, we encourage you to compare your work against the solution file to perform a self-assessment.

## Conditional Probability Model and Classification

In machine learning, we're often interested in a predictive modeling problem where we want to predict a class label for a given observation. For example, we might want to classify the species of plants based on the measurements of the flowers.

Problems of this type are referred to as classification predictive modeling problems. The observation or input to the model is referred to as `X`, and the class label or output of the model is referred to as `y`.

Together, `X` and `y` represent observations collected from the domain, i.e., a table or matrix (columns and rows or features and samples) of training data used to fit a model. The model must learn how to map specific examples to class labels or $y = f(X)$, which minimizes the misclassification.


One approach to solving this problem is to develop a **probabilistic model**. From a probabilistic perspective, we are interested in estimating the conditional probability of the class label, given the observation.

For example, a classification problem may have $k$ class labels $y_1, y_2, \dots, y_k$ and $n$ input variables, $X_1, X_2, \dots, X_n$. We can calculate the conditional probability for a class label with a given instance or set of input values for each column $X_1, X_2, \dots, X_n$ as:

$$P(y_i | X_1, X_2, \dots, X_n).$$


Bayes' theorem provides a principled way of calculating this conditional probability.

A simple form of the calculation for Bayes' theorem is as follows:

$$P(A|B) = P(B|A) * P(A) / P(B),$$

where the probability that we are interested in calculating, $P(A|B),$ is called the posterior probability, and the marginal probability of the event $P(A)$ is called the prior.

## Worked Example of Bayesian Probability

In this section, we will demonstrate the calculation using a small example on a machine learning dataset.

We can generate a small binary (2 class) classification problem using the [`make_blobs()`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) function from the `scikit-learn` library.


In the code cell below, we've imported the function `make_blobs` from `sklearn` and used that to generate 50 points with two numerical input variables, with each data point each assigned one of two classes. Modify the code cell below to generate 100 samples.

Notice that the `random_state` argument is set to 1, ensuring that the same random sample of observations is generated each time the code is run.

In [ ]:
# example of generating a small classification dataset
from sklearn.datasets import make_blobs

# generate classification dataset
X, y = make_blobs(n_samples=50, centers=2, n_features=2, random_state=1)

print('X shape:', X.shape)
print('y shape:', y.shape)

In the code cell below, we visualize the first five entries for `X` and `y`.

In [ ]:
# summarize
print(X[:5])
print(y[:5])

Above, the input and output elements of the first five examples are also printed, showing that, the two input variables are indeed numeric and the class labels are either 0 or 1 for each example.

## Modeling the Data


We will model the numerical input variables using a **Gaussian** (or "normal") probability distribution. 

<img src="gauss.png" alt="Drawing" style="width: 500px;"/>

This can be achieved by using the function `norm` from the `SciPy` library. `SciPy` is a free and open-source `Python` library used for scientific and technical computing. `SciPy` contains modules for optimization, linear algebra, integration, special functions, signal and image processing, and other tasks common in science and engineering.

The normal (Gaussian) probabililty distribution can be constructed by specifying the parameters of the distribution, i.e., the mean and standard deviation; the probability density function can then be sampled for specific values using the `norm.pdf()` function.

We can estimate the parameters of the distribution from the dataset using the `mean()` and `std()` `NumPy` functions.

In the code cell below, we've defined a custom-built function `fit_distribution()`, which takes a sample of data for one variable and returns a data distribution.

Run the code cell below.

In [ ]:
import numpy as np
from scipy.stats import norm

# fit a probability distribution to a univariate data sample
def fit_distribution(data):
    # estimate parameters
    mu = np.mean(data)
    sigma = np.std(data)
    print(mu, sigma)
    # fit distribution
    dist = norm(mu, sigma)
    return dist

Recall that we are interested in the conditional probability of each input variable. This means we need one distribution for each of the input variables and one set of distributions for each of the class labels or four distributions in total.

In the code cell below, we split the data into groups of samples for each of the class labels.

We start by separating the values of `X` that have corresponding `y` equal to 0.

In [ ]:
# sort data into 0 class
Xy0 = X[y == 0]

In the code cell below, define the variable `Xy1`, which contains those values of `X` for which `y` is equal to 1.

**HINT:** Follow the example in the cell above.

In [ ]:
# sort data into 1 class
Xy1 = X[y == ...]


Finally, we print the shape of the two sets we've created.

In [ ]:
print(Xy0.shape, Xy1.shape)

It'll also be useful to plot our data, with class labels 0 in red, and class labels 1 in black. We can see that each group looks approximately Gaussian, with good separation between the two groups. So there's hope in building a good prediction model!

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(Xy0[:,0], Xy0[:,1], c='red')  # class 0
plt.scatter(Xy1[:,0], Xy1[:,1], c='blue') # class 1
plt.show()

We can then use these groups to calculate the prior probabilities for a data sample belonging to each group.

This will be 50% exactly, given that we've created the same number of examples in each of the two classes; nevertheless, we will calculate these priors for completeness. We start with the prior for `Xy0`.

In [ ]:
# calculate priors
priory0 = len(Xy0) / len(X)

Following what we just did, in the code below calculate the prior for `Xy1`.

In [ ]:
priory1 = len(...) / len(...)

Below, we print the probabilities:

In [ ]:
print(priory0, priory1)

Finally, we can call the `fit_distribution()` function that we defined to prepare a probability distribution for each variable, for each class label. Run the code cell below:

In [ ]:
# create PDFs for y==0
distX1y0 = fit_distribution(Xy0[:, 0])
distX2y0 = fit_distribution(Xy0[:, 1])

In the code cell below, compute the PDFs parameters for `Xy1`.

**HINT:** Follow the example code above.

In [ ]:
# create PDFs for y==1
distX1y1 = fit_distribution(...[:, ...])
distX2y1 = fit_distribution(...[:, ...])


# Making a Prediction

Next, we can use the prepared probabilistic model to make a prediction.

The independent conditional probability for each class label can be calculated using the prior for the class (50%) and the conditional probability of the value for each variable.

The `probability()` function defined below performs this calculation. The value returned is a relative score rather than a probability, as the quantity is not normalized to $P(B)$ in Bayes' formula that normalization *is* handled by the corresponding sklearn function, which we will explore later below.

The function below takes four arguments:

- `X`: The input data
- `prior`: The prior distribution calculated above
- `dist1/dist2`: The conditional distribution for each variable

In [ ]:
# calculate the independent conditional probability
def probability(X, prior, dist1, dist2):
    return prior * dist1.pdf(X[0]) * dist2.pdf(X[1])

We can use this function to calculate the probability for an example belonging to each class.

First, we can select an example to be classified - in this case, the first example in the dataset.

In [ ]:
# classify one example
Xsample, ysample = X[0], y[0]

In the code cell below,  we can calculate the score of the example belonging to the first class.

In [ ]:
py0 = probability(Xsample, priory0, distX1y0, distX2y0)

Follow the code above to compute the probabily for the second class.

In [ ]:
py1 = probability(Xsample, priory1, distX1y1, distX2y1)

Next, we compute the probabilities:

In [ ]:
print('P(y=0 | %s) = %.3f' % (Xsample, py0*100))
print('P(y=1 | %s) = %.3f' % (Xsample, py1*100))

Based on the scores above, which will be the resulting classification?

**DOUBLE CLICK ON THIS CELL TO TYPE YOUR ANSWER**


## `sklearn` implementation

Of course, the `sklearn` library offers an implementation of the algorithm above.

It provides three implementations, one for each of the three main probability distributions; for example, `BernoulliNB`, `MultinomialNB`, and `GaussianNB` for binomial, multinomial, and Gaussian-distributed input variables, respectively.

To use a `sklearn`  Bayes' model, first, the model is defined, then it's fit on the training dataset. Once fit, probabilities can be predicted via the `predict_proba()` function, and class labels can be predicted directly via the `predict()` function.

The complete example of fitting a Gaussian Bayes' model (`GaussianNB`) to the same test dataset is given below.

In [ ]:
# example of gaussian naive bayes

#importing libraries
from sklearn.datasets import make_blobs
from sklearn.naive_bayes import GaussianNB
# generate 2d classification dataset
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [ ]:
# define the model
model = GaussianNB()
# fit the model
model.fit(X, y)
# select a single sample
Xsample, ysample = [X[0]], y[0]
# make a probabilistic prediction
yhat_prob = model.predict_proba(Xsample)

In [ ]:
print('Predicted Probabilities: ', yhat_prob)
# make a classification prediction
yhat_class = model.predict(Xsample)
print('Predicted Class: ', yhat_class)
print('Truth: y=%d' % ysample)

Running the example fits the model on the training dataset, then makes predictions for the same first example that we used in the prior example.

In [ ]:
print('Xsample:', Xsample)
print('Predicted Probabilities: ', yhat_prob)
# make a classification prediction
yhat_class = model.predict(Xsample)
print('Predicted Class: ', yhat_class)
print('True Class: y=%d' % ysample)

To help visualize this, we repeat the plot of our data but now adding the point whose class we are trying to predict, as a black 'X':

In [ ]:
plt.scatter(Xy0[:,0], Xy0[:,1], c='red')  #class 0
plt.scatter(Xy1[:,0], Xy1[:,1], c='blue') #class 1
plt.scatter(Xsample[0][0], Xsample[0][1], c='black', marker='X')
plt.show()

Let's try again, this time for a data point we haven't seen before:

In [ ]:
Xsample = np.array([[-9.0, -5.0]]) #new data point
print('Xsample:', Xsample)

# make a probabilistic prediction
yhat_prob = model.predict_proba(Xsample)

print('Predicted Probabilities: ', yhat_prob)
yhat_class = model.predict(Xsample)
print('Predicted Class: ', yhat_class)

plt.scatter(Xy0[:,0], Xy0[:,1], c='red')  #class 0
plt.scatter(Xy1[:,0], Xy1[:,1], c='blue') #class 1
plt.scatter(Xsample[0][0], Xsample[0][1], c='black', marker='X')
plt.show()